# CRS Prof Predictor

By:

Mariano, Isaiah

Montealto, Meluisa

Regalario, Jeremiah

In [76]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import xgboost as xgb

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Webscrape Data from CRS

## Links

In [77]:
first_sem = "https://crs.upd.edu.ph/schedule/1{}1/Math"
second_sem = "https://crs.upd.edu.ph/schedule/1{}2/Math"
midyr = "https://crs.upd.edu.ph/schedule/1{}4/Math"

## Enter Year Range

In [78]:
first_year = 2018
last_year = 2024

In [79]:
links = []
for i in range(first_year, last_year + 1):
  exec(f"AY{i}{i + 1}A = first_sem.format(i)")
  exec(f"AY{i}{i + 1}B = second_sem.format(i)")
  exec(f"AY{i}{i + 1}M = midyr.format(i)")
  exec(f"links.append(AY{i}{i + 1}A)")
  exec(f"links.append(AY{i}{i + 1}B)")
  exec(f"links.append(AY{i}{i + 1}M)")

In [80]:
AY20182019B

'https://crs.upd.edu.ph/schedule/120182/Math'

In [81]:
links

['https://crs.upd.edu.ph/schedule/120181/Math',
 'https://crs.upd.edu.ph/schedule/120182/Math',
 'https://crs.upd.edu.ph/schedule/120184/Math',
 'https://crs.upd.edu.ph/schedule/120191/Math',
 'https://crs.upd.edu.ph/schedule/120192/Math',
 'https://crs.upd.edu.ph/schedule/120194/Math',
 'https://crs.upd.edu.ph/schedule/120201/Math',
 'https://crs.upd.edu.ph/schedule/120202/Math',
 'https://crs.upd.edu.ph/schedule/120204/Math',
 'https://crs.upd.edu.ph/schedule/120211/Math',
 'https://crs.upd.edu.ph/schedule/120212/Math',
 'https://crs.upd.edu.ph/schedule/120214/Math',
 'https://crs.upd.edu.ph/schedule/120221/Math',
 'https://crs.upd.edu.ph/schedule/120222/Math',
 'https://crs.upd.edu.ph/schedule/120224/Math',
 'https://crs.upd.edu.ph/schedule/120231/Math',
 'https://crs.upd.edu.ph/schedule/120232/Math',
 'https://crs.upd.edu.ph/schedule/120234/Math',
 'https://crs.upd.edu.ph/schedule/120241/Math',
 'https://crs.upd.edu.ph/schedule/120242/Math',
 'https://crs.upd.edu.ph/schedule/120244

## Webscraping using BeautifulSoup

In [82]:
for link in links:
  page = requests.get(link)
  soup = BeautifulSoup(page.content, "html.parser")

  table = soup.find("table")
  headers = []
  for header in table.find_all('th'):
      headers.append(header.text.strip())
  rows = []
  for row in table.find_all('tr'):
      cells = row.find_all(['td', 'th'])
      cells = [cell.text.strip() for cell in cells]
      rows.append(cells)
  identifier = link.split("/")[4]
  exec("df{} = pd.DataFrame(rows, columns=headers)".format(identifier))
  exec("df{}['Year'] = identifier[1:5]".format(identifier)) #Start year
  exec("df{} = df{}.drop(df{}.index[0])".format(identifier, identifier, identifier))

In [83]:
df120231.head()

,Class Code,Class,Credits,Schedule\n\t\tInstructor(s)\n\t\tRemarks,Enlisting Unit : BlockBlock Remarks,Available Slots /\n\t\tTotal Slots,Demand,Restrictions,Year
1,39233,Math 2 THR,3.0,"TTh 8:30-10AM lec MBAN 306\n\t\t\tCONSORTE, OD...",MATH,0 /\n\t\t\t\t51,0,"For: BS HE(2 slots), AIT(3 slots)",2023
2,39235,Math 2 THU,3.0,"TTh 10-11:30AM lec MBAN 306\n\t\t\tAGUILAR, AD...",MATH,0 /\n\t\t\t\t52,0,,2023
3,39430,Math 2 THU-1,3.0,"TTh 10-11:30AM lec MBAN 312\n\t\t\tLUY, JASMIN...",MATH,0 /\n\t\t\t\t30,0,,2023
4,39234,Math 2 THW,3.0,"TTh 1-2:30PM lec MBAN 306\n\t\t\tABARA, MA. NE...",MATH,0 /\n\t\t\t\t56,0,,2023
5,39242,Math 2 WFU,3.0,"WF 10-11:30AM lec MBAN 306\n\t\t\tGALVEZ, LUCK...",MATH,0 /\n\t\t\t\t51,0,"For: BA Film(1 slots), BA BMAS(1 slots), SLIS(...",2023


In [84]:
len(df120231)

303

## Concatenate DataFrames

In [85]:
for link in links[1:]:
  identifier = link.split("/")[4]
  exec("df120181 = pd.concat([df120181, df{}])".format(identifier))
df = df120181.copy()

In [86]:
len(df)

3902

# Data Cleaning

In [87]:
df.head()

,Class Code,Class,Credits,Schedule\n\t\tInstructor(s)\n\t\tRemarks,Enlisting Unit : BlockBlock Remarks,Available Slots /\n\t\tTotal Slots,Demand,Restrictions,Year
1,39236,Math 2 THR,3.0,"TTh 8:30-10AM lec TBA\n\t\t\tWALO, MA. LAILANI...",MATH,1 /\n\t\t\t\t64,0,,2018
2,39398,Math 2 THU,3.0,"TTh 10-11:30AM lec TBA\n\t\t\tCEJO, ROBERT JAY...",MATH,0 /\n\t\t\t\t59,0,,2018
3,39238,Math 2 THV,3.0,"TTh 11:30AM-1PM lec TBA\n\t\t\tWALO, MA. LAILA...",MATH,0 /\n\t\t\t\t63,0,,2018
4,39237,Math 2 WFV,3.0,"WF 11:30AM-1PM lec TBA\n\t\t\tAGUILAR, ADJANI\...",MATH,0 /\n\t\t\t\t65,0,,2018
5,39315,Math 10 THR,3.0,"TTh 8:30-10AM lec TBA\n\t\t\tARCEO, CARLENE PE...",MATH,0 /\n\t\t\t\t97,0,,2018


## Selected Columns

In [88]:
df = df[['Class', 'Schedule\n\t\tInstructor(s)\n\t\tRemarks', 'Year']]
df.head()

,Class,Schedule\n\t\tInstructor(s)\n\t\tRemarks,Year
1,Math 2 THR,"TTh 8:30-10AM lec TBA\n\t\t\tWALO, MA. LAILANI...",2018
2,Math 2 THU,"TTh 10-11:30AM lec TBA\n\t\t\tCEJO, ROBERT JAY...",2018
3,Math 2 THV,"TTh 11:30AM-1PM lec TBA\n\t\t\tWALO, MA. LAILA...",2018
4,Math 2 WFV,"WF 11:30AM-1PM lec TBA\n\t\t\tAGUILAR, ADJANI\...",2018
5,Math 10 THR,"TTh 8:30-10AM lec TBA\n\t\t\tARCEO, CARLENE PE...",2018


## Math Class Number

In [89]:
df['Number'] = df['Class'].str.split().str[1]
df.head()

,Class,Schedule\n\t\tInstructor(s)\n\t\tRemarks,Year,Number
1,Math 2 THR,"TTh 8:30-10AM lec TBA\n\t\t\tWALO, MA. LAILANI...",2018,2
2,Math 2 THU,"TTh 10-11:30AM lec TBA\n\t\t\tCEJO, ROBERT JAY...",2018,2
3,Math 2 THV,"TTh 11:30AM-1PM lec TBA\n\t\t\tWALO, MA. LAILA...",2018,2
4,Math 2 WFV,"WF 11:30AM-1PM lec TBA\n\t\t\tAGUILAR, ADJANI\...",2018,2
5,Math 10 THR,"TTh 8:30-10AM lec TBA\n\t\t\tARCEO, CARLENE PE...",2018,10


In [90]:
df['Number'].unique()

array(['2', '10', '11', '14', '17', '20', '21', '30', '40', '53', '54',
       '55', '100', '102', '109', '110.1', '110.2', '110.3', '117', '121',
       '122', '123.1', '123.2', '128', '147', '150.1', '162', '164',
       '171', '180.1', '197', '200', '201', '202.1', '203', '205', '208',
       '209.1', '210.1', '211', '214', '220.1', '221', '222', '228',
       '229', '236', '240', '242', '243', '246', '260', '262.1', '265',
       '271.1', '280', '290', '294', '296', '297', '300', '400', '22',
       '108', '140', '142', '146', '148', '150.2', '196', '202.2', '204',
       '209.2', '210.2', '218', '220.2', '235', '241', '247', '250',
       '261', '262.2', '266', '271.2', '281', '295', '23', '217', '288',
       '249', '216', '133', '158', '190', '227', '126', '166', '282',
       '131', '258', '/', None], dtype=object)

## Remove / and None

In [91]:
df = df[df['Number'] != '/']
len(df)

3885

In [92]:
df = df[df['Number'].isna() == False]
len(df)

3883

## Create Day, Time, and Prof Columns

In [93]:
# Function to extract day, time, and professor
def extract_info(schedule):
    day_pattern = r'\b(M|TTh|WF|S|TWThF)\b'
    time_pattern = r'\b\d{1,2}(?::\d{2})?(?:AM|PM)?-\d{1,2}(?::\d{2})?(?:AM|PM)?\b'

    prof_pattern = r'(?<=\n\t\t\t).*$'
    prof_match = re.search(prof_pattern, schedule, re.MULTILINE | re.DOTALL)
    prof = prof_match.group().strip() if prof_match else ''

    day = re.search(day_pattern, schedule)
    time = re.search(time_pattern, schedule)


    return (
        day.group() if day else '',
        time.group() if time else '',
        prof
    )

# Apply the function to the DataFrame
df[['Day', 'Time', 'Prof']] = df['Schedule\n\t\tInstructor(s)\n\t\tRemarks'].apply(lambda x: pd.Series(extract_info(x)))

In [94]:
df['Prof'] = df['Prof'].str.split('\n').str[0]

In [95]:
df = df.dropna(subset=['Prof'])  # Drop rows where 'Prof' is NaN
df = df.dropna(subset=['Prof'])  # Drop rows where 'Time' is NaN
df = df[~df['Prof'].str.upper().isin(['CONCEALED', 'TBA'])]

In [96]:
df = df[["Number", "Day", "Time", "Prof", "Year"]]

In [97]:
df

,Number,Day,Time,Prof,Year
1,2,TTh,8:30-10AM,"WALO, MA. LAILANI",2018
2,2,TTh,10-11:30AM,"CEJO, ROBERT JAY",2018
3,2,TTh,11:30AM-1PM,"WALO, MA. LAILANI",2018
4,2,WF,11:30AM-1PM,"AGUILAR, ADJANI",2018
5,10,TTh,8:30-10AM,"ARCEO, CARLENE PERPETUA",2018
6,10,TTh,10-11:30AM,"ARCEO, CARLENE PERPETUA",2018
7,10,TTh,1-2:30PM,"NERI, MARRICK",2018
8,10,WF,10-11:30AM,"BALMACEDA, JOSE MARIA",2018
9,11,TTh,11:30AM-1PM,"VERGARA, THOMAS HERALD",2018
10,11,WF,8:30-10AM,"RIVERO, RACHELLE; VERGARA, THOMAS HERALD",2018


In [98]:
df[df['Prof'] == 'AGAPITO, JULIUS CAESAR']

,Number,Day,Time,Prof,Year
140,53,TWThF,8:30-9:45AM,"AGAPITO, JULIUS CAESAR",2018
179,121,WF,2:30-4PM,"AGAPITO, JULIUS CAESAR",2018
229,229,WF,4-5:30PM,"AGAPITO, JULIUS CAESAR",2018
132,55,TTh,1-2:30PM,"AGAPITO, JULIUS CAESAR",2018
150,122,TTh,11:30AM-1PM,"AGAPITO, JULIUS CAESAR",2018
152,123.2,WF,11:30AM-1PM,"AGAPITO, JULIUS CAESAR",2018
220,220.2,TTh,1-2:30PM,"AGAPITO, JULIUS CAESAR",2018
227,242,WF,4-5:30PM,"AGAPITO, JULIUS CAESAR",2018
270,220.1,WF,4-5:30PM,"AGAPITO, JULIUS CAESAR",2019
275,229,TTh,2:30-4PM,"AGAPITO, JULIUS CAESAR",2019


In [99]:
df[df['Prof'].str.contains('AGAPITO, JULIUS CAESAR')]

,Number,Day,Time,Prof,Year
140,53,TWThF,8:30-9:45AM,"AGAPITO, JULIUS CAESAR",2018
179,121,WF,2:30-4PM,"AGAPITO, JULIUS CAESAR",2018
229,229,WF,4-5:30PM,"AGAPITO, JULIUS CAESAR",2018
132,55,TTh,1-2:30PM,"AGAPITO, JULIUS CAESAR",2018
150,122,TTh,11:30AM-1PM,"AGAPITO, JULIUS CAESAR",2018
152,123.2,WF,11:30AM-1PM,"AGAPITO, JULIUS CAESAR",2018
220,220.2,TTh,1-2:30PM,"AGAPITO, JULIUS CAESAR",2018
222,220.2,TTh,5:30-7PM,"AGAPITO, JULIUS CAESAR; SY, POLLY",2018
227,242,WF,4-5:30PM,"AGAPITO, JULIUS CAESAR",2018
270,220.1,WF,4-5:30PM,"AGAPITO, JULIUS CAESAR",2019


## Convert to Military Time

In [100]:
def to_military_time(time, suffix):
    if "AM" not in time and "PM" not in time:
        if ":" in time:
            hour, minute = time.split(":")
            hour = int(hour)
            minute = int(minute)
            if suffix == "PM" and hour < 8:
                hour += 12
            return f"{hour:02}:{minute:02}"
        else:
            try:
              time = int(time)
              if suffix == "PM" and time < 8:
                time += 12
              return f"{time:02}:00"
            except:
              return f"e: {time} 1"
    elif time[-2:] == "AM":
        time = time[:-2]
        if ":" in time:
            hour, minute = time.split(":")
            hour = int(hour)
            minute = int(minute)
            return f"{hour:02}:{minute:02}"
        else:
            try:
              time = int(time)
              return f"{time:02d}:00"
            except:
              return f"e: {time} 2"
    elif time[-2:] == "PM":
        time = time[:-2]
        if ":" in time:
            hour, minute = time.split(":")
            hour = int(hour)
            if hour < 8:
              hour += 12
            minute = int(minute)
            return f"{hour:02}:{minute:02}"
        else:
            try:
              time = int(time)
              if time < 8:
                time += 12
              return f"{time:02}:00"
            except:
              return f"e: {time} 3"

def split_time(time):
    time = str(time)
    if "-" not in time:
        return "", ""
    start_time, end_time = time.split('-')
    start_time = str(start_time)
    end_time = str(end_time)
    suffix = time[-2:]
    return to_military_time(start_time, suffix), to_military_time(end_time, suffix)

In [101]:
df["Time"].unique()

array(['8:30-10AM', '10-11:30AM', '11:30AM-1PM', '1-2:30PM', '2:30-4PM',
       '7-8:30AM', '2:30-3:45PM', '7:15-8:15AM', '8:45-9:45AM',
       '1:15-2:15PM', '2:45-3:45PM', '10:15-11:15AM', '11:45AM-12:45PM',
       '8:30-9:45AM', '1-2:15PM', '10-11:15AM', '11:30AM-12:45PM',
       '4-5:30PM', '12-3PM', '8:30-11:30AM', '', '5:30-7PM', '10AM-1PM',
       '4-7PM', '9AM-12PM', '7-10AM', '3-4PM', '4-5PM', '1-4PM',
       '4:15-5:15PM', '3-6PM', '7-9AM', '9-11AM', '11AM-1PM',
       '9:35AM-12:10PM', '12:10-2:45PM', '2:45-5:20PM', '7-9:35AM',
       '10:15AM-1:30PM', '1-3PM', '7-8:15AM', '5-8PM', '2-5PM', '12-1PM',
       '1-2PM', '10-11AM', '2:30-5:30PM', '8-11AM', '11:30AM-2:30PM',
       '9-10AM', '11AM-12:15PM', '3-4:15PM', '9-10:15AM', '2:45-4:30PM',
       '7-8:45AM', '9:35-11:20AM', '12:10-1:55PM', '5-6:15PM', '6-9PM',
       '4-6PM', '7:30-9:30AM', '10:30AM-12:30PM', '6-7PM', '5-6PM',
       '3-5PM'], dtype=object)

In [102]:
df[["Start_time", "End_time"]] = df["Time"].apply(lambda x: pd.Series(split_time(x)))

In [103]:
df

,Number,Day,Time,Prof,Year,Start_time,End_time
1,2,TTh,8:30-10AM,"WALO, MA. LAILANI",2018,08:30,10:00
2,2,TTh,10-11:30AM,"CEJO, ROBERT JAY",2018,10:00,11:30
3,2,TTh,11:30AM-1PM,"WALO, MA. LAILANI",2018,11:30,13:00
4,2,WF,11:30AM-1PM,"AGUILAR, ADJANI",2018,11:30,13:00
5,10,TTh,8:30-10AM,"ARCEO, CARLENE PERPETUA",2018,08:30,10:00
6,10,TTh,10-11:30AM,"ARCEO, CARLENE PERPETUA",2018,10:00,11:30
7,10,TTh,1-2:30PM,"NERI, MARRICK",2018,13:00,14:30
8,10,WF,10-11:30AM,"BALMACEDA, JOSE MARIA",2018,10:00,11:30
9,11,TTh,11:30AM-1PM,"VERGARA, THOMAS HERALD",2018,11:30,13:00
10,11,WF,8:30-10AM,"RIVERO, RACHELLE; VERGARA, THOMAS HERALD",2018,08:30,10:00


In [104]:
df.dtypes

,0
Number,object
Day,object
Time,object
Prof,object
Year,object
Start_time,object
End_time,object


## Remove Original Time column

In [105]:
df = df.drop(columns=["Time"])
df.head()

,Number,Day,Prof,Year,Start_time,End_time
1,2,TTh,"WALO, MA. LAILANI",2018,08:30,10:00
2,2,TTh,"CEJO, ROBERT JAY",2018,10:00,11:30
3,2,TTh,"WALO, MA. LAILANI",2018,11:30,13:00
4,2,WF,"AGUILAR, ADJANI",2018,11:30,13:00
5,10,TTh,"ARCEO, CARLENE PERPETUA",2018,08:30,10:00


# Random Forest

## Convert Time

In [106]:
model_df = df.copy()

In [107]:
model_df['Start_time'] = pd.to_datetime(model_df['Start_time']).dt.hour * 60 + pd.to_datetime(model_df['Start_time']).dt.minute
model_df['End_time'] = pd.to_datetime(model_df['End_time']).dt.hour * 60 + pd.to_datetime(model_df['End_time']).dt.minute
model_df['Year'] = model_df['Year'].astype(int)
model_df.head()

<ipython-input-107-a0d962557279>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  model_df['Start_time'] = pd.to_datetime(model_df['Start_time']).dt.hour * 60 + pd.to_datetime(model_df['Start_time']).dt.minute
<ipython-input-107-a0d962557279>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  model_df['Start_time'] = pd.to_datetime(model_df['Start_time']).dt.hour * 60 + pd.to_datetime(model_df['Start_time']).dt.minute
<ipython-input-107-a0d962557279>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  model_df['End_time'] = pd.to_datetime(model_df['End_time']).dt.hour * 60 + pd.

,Number,Day,Prof,Year,Start_time,End_time
1,2,TTh,"WALO, MA. LAILANI",2018,510.0,600.0
2,2,TTh,"CEJO, ROBERT JAY",2018,600.0,690.0
3,2,TTh,"WALO, MA. LAILANI",2018,690.0,780.0
4,2,WF,"AGUILAR, ADJANI",2018,690.0,780.0
5,10,TTh,"ARCEO, CARLENE PERPETUA",2018,510.0,600.0


## Drop Null Values

In [108]:
model_df = model_df.dropna()
len(df)

2709

In [109]:
len(model_df)

2261

In [110]:
# Encode categorical variables
model_df = pd.get_dummies(model_df, columns=['Day', 'Number'])

# Encode the target variable 'Prof'
le = LabelEncoder()
model_df['Prof'] = le.fit_transform(model_df['Prof'])

# Split the data into features and target
X = model_df.drop(columns=['Prof'])
y = model_df['Prof']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
model = RandomForestClassifier() #TO DO: Hyperparameter Tuning
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.15


In [111]:
# Decode the predicted labels back to original professor names (optional)
y_pred_labels = le.inverse_transform(y_pred)

# Example prediction
def predict_prof(number, day, start_time, end_time):

    # Convert time to minutes
    start_minutes = pd.to_datetime(start_time).hour * 60 + pd.to_datetime(start_time).minute
    end_minutes = pd.to_datetime(end_time).hour * 60 + pd.to_datetime(end_time).minute

    # Create a DataFrame for the input
    input_df = pd.DataFrame({
        'Start_time': [start_minutes],
        'End_time': [end_minutes],
        f'Day_{day}': [1],
        f'Number_{number}': [1]
    })

    # Add missing columns with zeros
    for col in X.columns:
        if col not in input_df.columns:
            input_df[col] = 0

    # Predict the professor
    pred = model.predict(input_df[X.columns])
    return le.inverse_transform(pred)[0]

# Prof Predictor (Use Military Time)

In [135]:
codes = {
    "R": ("8:30", "10:00"),
    "U": ("10:00", "11:30"),
    "V": ("11:30", "13:00"),
    "W": ("13:00", "14:30"),
    "X": ("14:30", "16:00"),
    "Y": ("16:00", "17:30")
}

def convert_class(name):
  math, number, code = name.split()
  return number, code[:2], *codes[code[2]]

In [114]:
predicted_prof = predict_prof(123.2, 'WF', '14:30', '16:00')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['CABARRUBIAS, BITUIN']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [116]:
predicted_prof = predict_prof(123.2, 'WF', '13:00', '14:30')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['CABARRUBIAS, BITUIN']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [117]:
predicted_prof = predict_prof(162, 'WF', '13:00', '14:30')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['RAMOS, AARON']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [118]:
predicted_prof = predict_prof(126, 'WF', '13:00', '14:30')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['DELA CRUZ, ROMAR; NERI, MARRICK']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [113]:
predicted_prof = predict_prof(150.1, 'WF', '10:30', '11:30')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['MARFIL, RAMON IÑIGO']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [119]:
predicted_prof = predict_prof(150.1, 'WF', '13:00', '14:30')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['ESCANER, JOSE MARIA']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [120]:
predicted_prof = predict_prof(171, 'WF', '16:00', '17:30')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['RAMOS, AARON']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [121]:
predicted_prof = predict_prof(171, 'TH', '16:00', '17:30')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['RAMOS, AARON']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 

In [122]:
predicted_prof = predict_prof(128, 'WF', '11:30', '13:00')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['ONA, MARK PHILIP']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [123]:
predicted_prof = predict_prof(108, 'WF', '11:30', '13:00')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['DAMASCO, JOSEPH RAY CLARENCE']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [124]:
predicted_prof = predict_prof(142, 'TH', '11:30', '13:00')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['VENTURA, JADE']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 

In [125]:
predicted_prof = predict_prof(142, 'WF', '14:30', '16:00')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['AGAPITO, JULIUS CAESAR']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [126]:
predicted_prof = predict_prof(142, 'WF', '11:30', '13:00')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['DE LA CRUZ, RALPH JOHN']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [133]:
predicted_prof = predict_prof(110.1, 'WF', '08:30', '10:00')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['BETTY, ROWENA ALMA']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [128]:
predicted_prof = predict_prof(110.1, 'WF', '10:00', '11:30')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['BETTY, ROWENA ALMA']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [129]:
predicted_prof = predict_prof(140, 'TH', '11:30', '13:00')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['JUNIO, ALLAN']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 

In [130]:
predicted_prof = predict_prof(140, 'TH', '13:00', '14:30')
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['CANLUBO, CLARISSON RIZZIE']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 

In [138]:
class_name = "Math 122 WFX"
predicted_prof = predict_prof(*convert_class(class_name))
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['CABARRUBIAS, BITUIN']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [139]:
class_name = "Math 122 WFW"
predicted_prof = predict_prof(*convert_class(class_name))
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['CABARRUBIAS, BITUIN']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [140]:
class_name = "Math 122 WFU"
predicted_prof = predict_prof(*convert_class(class_name))
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['DE LOS REYES, AURELIO']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [141]:
class_name = "Math 122 WFR"
predicted_prof = predict_prof(*convert_class(class_name))
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['DE LOS REYES, AURELIO']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0


In [143]:
class_name = "Math 122 THX"
predicted_prof = predict_prof(*convert_class(class_name))
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['CABARRUBIAS, BITUIN']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 

In [145]:
class_name = "Math 122 THW"
predicted_prof = predict_prof(*convert_class(class_name))
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['CABARRUBIAS, BITUIN']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 

In [146]:
class_name = "Math 122 THV"
predicted_prof = predict_prof(*convert_class(class_name))
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['AGAPITO, JULIUS CAESAR']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 

In [147]:
class_name = "Math 122 THU"
predicted_prof = predict_prof(*convert_class(class_name))
print('Predicted Professor:', predicted_prof)

Predicted Professor: ['PAGUIO, VICTORIA MAY']


<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0
<ipython-input-111-65f087b39db5>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 